In [1]:
import pandas as pd
import numpy as np
import re

## Neighborhood Safety Index Data Clean

In [2]:
## Import all the datasets
svi = pd.read_csv('Original_Datasets/SVI-2022.csv')
epa = pd.read_csv('Original_Datasets/EPA_Smart_location.csv')
ejs = pd.read_csv('Original_Datasets/EJS_Variables.csv')
vacancyhouse = pd.read_csv('Original_Datasets/ACS/vacancy_house.csv')
medicaid = pd.read_csv('Original_Datasets/ACS/Medicaid_coverage.csv')
transportationmode = pd.read_csv('Original_Datasets/ACS/Mode_Transportation_Work.csv')
snap = pd.read_csv('Original Datasets/ACS/SNAP.csv')
ssi = pd.read_csv('Original Datasets/ACS/SSI.csv')
traveltime = pd.read_csv('Original Datasets/ACS/Traveltime_Work.csv')
housingcost = pd.read_csv('Original Datasets/ACS/housing_cost.csv')
populationoccupied = pd.read_csv('Original Datasets/ACS/Population_occupiedhousing.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Original Datasets/SVI-2022.csv'

### EJScreen Dataset

In [3]:
ejs.head()

,ID,STATE_NAME,ST_ABBREV,CNTY_NAME,ACSTOTPOP,ACSIPOVBAS,ACSEDUCBAS,ACSTOTHH,ACSTOTHU,ACSUNEMPBAS,...,PRE1960PCT,PNPL,PRMP,PTSDF,UST,PWDIS,NO2,DWATER,Shape_Length,Shape_Area
0,51153900100,VIRGINIA,VA,Prince William County,3234,3234,2760,1559,1613,1670,...,0.045257,0.150439,0.199625,0.535742,0.552918,2955.696985,5.941893,0.0,0.177784,0.001610
1,51153900201,VIRGINIA,VA,Prince William County,2441,2436,1480,961,1058,1587,...,0.112476,0.107441,0.212287,0.506328,5.158356,1608.187692,7.430988,0.0,0.066764,0.000169
2,51153900202,VIRGINIA,VA,Prince William County,4124,4124,2770,1125,1266,2557,...,0.584518,0.100379,0.197937,0.323682,2.421485,1481.485726,7.024272,0.0,0.064834,0.000207
3,51153900203,VIRGINIA,VA,Prince William County,4589,4589,2781,1406,1616,2237,...,0.102104,0.108553,0.000000,0.257902,1.432822,1696.977990,7.046776,0.0,0.045018,0.000111
4,51153900301,VIRGINIA,VA,Prince William County,3494,3494,2526,1503,1610,2072,...,0.198137,0.039346,0.235448,0.455012,5.261958,749.776736,6.979262,0.0,0.072386,0.000198


In [4]:
ejs.shape

(93, 46)

In [11]:
# Select variables we would like to use (use tract ID as the common key)
df = ejs[['ID','PEOPCOLORPCT','LOWINCPCT','UNEMPPCT','DISABILITYPCT','LINGISOPCT','LESSHSPCT','UNDER5PCT','OVER64PCT','LIFEEXPPCT','PRE1960PCT','PM25','OZONE','DSLPM',\
 'RSEI_AIR','PTRAF','PTSDF','UST','PWDIS','NO2','DWATER','PNPL','PRMP']]
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,DSLPM,RSEI_AIR,PTRAF,PTSDF,UST,PWDIS,NO2,DWATER,PNPL,PRMP
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.170222,58.795632,1048284.539,0.535742,0.552918,2955.696985,5.941893,0.0,0.150439,0.199625
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.246331,47.188314,1939166.587,0.506328,5.158356,1608.187692,7.430988,0.0,0.107441,0.212287
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.243887,42.405394,1942872.958,0.323682,2.421485,1481.485726,7.024272,0.0,0.100379,0.197937
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.217924,45.322696,1432192.241,0.257902,1.432822,1696.977990,7.046776,0.0,0.108553,0.000000
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.250597,42.574405,1980590.732,0.455012,5.261958,749.776736,6.979262,0.0,0.039346,0.235448


In [13]:
# Check missing values
df.isna().sum()

ID                0
PEOPCOLORPCT      0
LOWINCPCT         0
UNEMPPCT          0
DISABILITYPCT     0
LINGISOPCT        0
LESSHSPCT         0
UNDER5PCT         0
OVER64PCT         0
LIFEEXPPCT       16
PRE1960PCT        0
PM25              0
OZONE             0
DSLPM             0
RSEI_AIR          0
PTRAF             0
PTSDF             0
UST               0
PWDIS             0
NO2               0
DWATER            3
PNPL              0
PRMP              0
dtype: int64

In [16]:
# Check description
df[['UNDER5PCT','OVER64PCT','LIFEEXPPCT','PRE1960PCT','PM25','OZONE','DSLPM',\
 'RSEI_AIR','PTRAF','PTSDF','UST','PWDIS','NO2','DWATER','PNPL','PRMP']].describe()

,UNDER5PCT,OVER64PCT,LIFEEXPPCT,PRE1960PCT,PM25,OZONE,DSLPM,RSEI_AIR,PTRAF,PTSDF,UST,PWDIS,NO2,DWATER,PNPL,PRMP
count,93.000000,93.000000,77.000000,93.000000,93.000000,93.000000,93.000000,93.000000,9.300000e+01,93.000000,93.000000,93.000000,93.000000,90.0,93.000000,93.000000
mean,0.067499,0.106965,0.178395,0.054362,7.171720,54.883629,0.163376,35.039448,7.885164e+05,0.199074,1.439193,2871.172997,5.674355,0.0,0.518651,0.092234
std,0.029585,0.085782,0.019857,0.091405,0.096046,0.732919,0.035009,62.611596,4.125723e+05,0.235906,1.455113,3788.676778,1.112318,0.0,1.982865,0.173471
min,0.000000,0.000000,0.117949,0.000000,6.941975,53.011480,0.087736,4.432628,1.190745e+05,0.000000,0.000000,22.112306,2.257740,0.0,0.000000,0.000000
25%,0.048142,0.064617,0.169231,0.005721,7.117304,54.385850,0.142895,18.634662,4.695638e+05,0.042028,0.274337,157.425225,5.111574,0.0,0.000000,0.000000
50%,0.067797,0.088678,0.175385,0.022654,7.189668,54.984990,0.159426,28.894226,7.313580e+05,0.171660,1.027627,2152.487718,5.647360,0.0,0.103988,0.000000
75%,0.083062,0.115750,0.185641,0.051193,7.241790,55.429870,0.178841,39.155226,9.792501e+05,0.265225,2.193942,3670.781811,6.351480,0.0,0.199042,0.116598
max,0.174067,0.640244,0.226667,0.584518,7.341178,56.036260,0.250597,614.114539,1.980591e+06,1.759346,7.353389,27419.695530,8.391446,0.0,11.192446,1.217279


In [103]:
df.head(10)

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P,With_SSI_P,Mean_Transportation_time(min),Mean_Proportion_HHIncome,Owner_occupied_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526,0.053239,45.510671,0.236878,0.840136
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125,0.037461,40.846185,0.353119,0.530788
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667,0.000000,35.776432,0.258486,0.841901
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102,0.077525,39.437598,0.339889,0.101680
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490,0.023287,37.584449,0.345896,0.603892
5,51153900302,0.697602,0.289381,0.008594,0.098866,0.066008,0.171859,0.048142,0.064617,0.180513,...,0.066118,0.058772,0.000000,0.022039,0.165736,0.055680,0.017063,43.303980,0.351252,0.537251
6,51153900403,0.752579,0.254023,0.054405,0.103629,0.103077,0.151957,0.127231,0.125921,0.160000,...,0.239325,0.039391,0.003641,0.004303,0.083416,0.166154,0.060000,36.821043,0.489604,0.828097
7,51153900404,0.850228,0.354006,0.089608,0.081358,0.199541,0.218959,0.052087,0.081024,0.164103,...,0.124066,0.032511,0.016442,0.032885,0.099776,0.130161,0.029243,47.328909,0.375236,0.710951
8,51153900407,0.823247,0.276344,0.069093,0.087767,0.071365,0.146658,0.097898,0.087318,NaN,...,0.054845,0.024027,0.004440,0.014886,0.179420,0.120567,0.026596,43.944940,0.321159,0.446748
9,51153900408,0.674728,0.068616,0.033874,0.081798,0.091917,0.101277,0.131984,0.069554,0.178462,...,0.114519,0.024457,0.009705,0.009705,0.130823,0.129074,0.004563,45.270210,0.258562,0.799400


### EPA Smart Location Dataset

In [110]:
epa.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
15824,15825,5.115390e+11,5.115390e+11,51,153,900901,3,548.0,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900.0,...,-99999.000000,-99999.000000,-99999.000000,3,3,11,1,5.000000,3288.378513,4.882644e+05
15832,15833,5.115390e+11,5.115390e+11,51,153,901223,2,548.0,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900.0,...,0.017243,0.000096,0.055578,8,11,13,13,11.833333,4659.655611,9.246838e+05
15835,15836,5.115390e+11,5.115390e+11,51,153,900600,2,548.0,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900.0,...,0.018999,0.000066,0.037985,12,13,9,14,11.833333,6006.277371,6.551407e+05
15836,15837,5.115390e+11,5.115390e+11,51,153,901001,2,548.0,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900.0,...,0.001148,0.000003,0.001931,15,13,9,13,12.000000,6508.049700,1.691396e+06
15837,15838,5.115390e+11,5.115390e+11,51,153,901001,1,548.0,"Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",47900.0,...,-99999.000000,-99999.000000,-99999.000000,17,16,10,1,9.166667,4913.735039,1.404021e+06


In [135]:
epa.shape

(230, 118)

In [148]:
# Select variables
epa1 = epa[['TRACTCE','BLKGRPCE','Ac_Total','Ac_Water','Ac_Land','Ac_Unpr','D1A','D1B','D1C','D1D','D2A_JPHH','D3A','D3B','D4A','D4C',\
            'D4D','D4E','NatWalkInd']]
epa1.head()

,TRACTCE,BLKGRPCE,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,D1A,D1B,D1C,D1D,D2A_JPHH,D3A,D3B,D4A,D4C,D4D,D4E,NatWalkInd
15824,900901,3,120.650034,0.231780,120.418255,120.418255,5.921029,15.886296,0.290654,6.211683,0.054859,11.834239,62.029134,-99999.00,1.33,7.068696,0.000695,5.000000
15832,901223,2,228.489664,0.000000,228.489664,228.489664,3.580031,10.451239,0.905949,4.485980,0.277852,19.884924,84.058069,846.25,1.33,3.725333,0.000557,11.833333
15835,900600,2,161.885835,2.474706,159.411129,146.000280,2.712324,15.493121,1.089039,3.801363,0.401515,12.053941,46.860467,706.77,9.33,37.457862,0.004125,11.833333
15836,901001,2,417.944199,0.550786,417.393413,417.393410,1.550096,3.991438,2.592279,4.142375,1.672334,16.537683,43.972711,922.69,2.67,4.093979,0.001603,12.000000
15837,901001,1,346.933589,1.170266,345.763324,341.308300,1.394634,4.175111,1.552848,2.947482,1.188341,14.300312,56.786127,-99999.00,8.00,14.807817,0.005614,9.166667


In [150]:
epa1.describe()

,TRACTCE,BLKGRPCE,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,D1A,D1B,D1C,D1D,D2A_JPHH,D3A,D3B,D4A,D4C,D4D,D4E,NatWalkInd
count,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,96.000000,157.000000,157.000000,157.000000,230.000000
mean,901477.430435,2.060870,967.397772,33.172121,934.225651,828.601339,2.374700,7.161570,1.170286,3.544986,1.013253,13.541535,52.729920,686.681250,4.365096,12.284386,0.002477,8.621014
std,5223.103801,1.047242,2063.245375,122.321740,2045.956811,1773.404290,2.038423,5.618399,2.099037,3.128898,3.009207,6.139447,38.636193,304.928481,3.789461,13.926169,0.002157,3.601878
min,900100.000000,1.000000,44.555836,0.000000,44.555836,41.455460,0.000000,0.000000,0.002279,0.002279,0.000000,2.051457,1.743151,84.490000,0.670000,0.046663,0.000000,1.666667
25%,901001.000000,1.000000,181.962340,0.000000,175.926737,167.347896,0.964913,2.871064,0.113022,1.585305,0.083784,8.793749,27.161990,476.767500,1.330000,2.912426,0.000774,5.500000
50%,901226.500000,2.000000,345.929622,1.365228,345.344489,321.757682,1.910230,6.506676,0.330714,2.751236,0.240980,13.069439,45.399913,698.720000,3.670000,6.294323,0.001747,8.000000
75%,901413.750000,3.000000,918.221438,10.808586,839.153391,713.944917,3.129426,9.901565,1.332196,4.601315,0.756776,17.148406,67.862908,925.037500,5.670000,17.880133,0.003808,11.333333
max,980100.000000,6.000000,20984.618590,1420.318198,20937.170450,19305.346370,11.625407,37.638374,16.128475,18.405296,28.753316,39.032837,232.641921,1188.230000,19.670000,88.554283,0.012544,19.500000


We noticed some values are -99999, so they are probably missing values, we replace them with NA.

In [ ]:
# List of columns contains -99999
columns_to_fix = ['D4A', 'D4C', 'D4D', 'D4E']

# Convert the columns to numeric
epa1[columns_to_fix] = epa1[columns_to_fix].apply(pd.to_numeric, errors='coerce')

# Replace -99999 with pd.NA in selected columns
for col in columns_to_fix:
    epa1[col] = epa1[col].mask(epa1[col] == -99999, pd.NA)

In [151]:
epa1.head()

,TRACTCE,BLKGRPCE,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,D1A,D1B,D1C,D1D,D2A_JPHH,D3A,D3B,D4A,D4C,D4D,D4E,NatWalkInd
15824,900901,3,120.650034,0.231780,120.418255,120.418255,5.921029,15.886296,0.290654,6.211683,0.054859,11.834239,62.029134,NaN,1.33,7.068696,0.000695,5.000000
15832,901223,2,228.489664,0.000000,228.489664,228.489664,3.580031,10.451239,0.905949,4.485980,0.277852,19.884924,84.058069,846.25,1.33,3.725333,0.000557,11.833333
15835,900600,2,161.885835,2.474706,159.411129,146.000280,2.712324,15.493121,1.089039,3.801363,0.401515,12.053941,46.860467,706.77,9.33,37.457862,0.004125,11.833333
15836,901001,2,417.944199,0.550786,417.393413,417.393410,1.550096,3.991438,2.592279,4.142375,1.672334,16.537683,43.972711,922.69,2.67,4.093979,0.001603,12.000000
15837,901001,1,346.933589,1.170266,345.763324,341.308300,1.394634,4.175111,1.552848,2.947482,1.188341,14.300312,56.786127,NaN,8.00,14.807817,0.005614,9.166667


In [152]:
# Transform block-level variables to tract-level
epa2 = epa1.groupby('TRACTCE').agg({
    'Ac_Total': 'sum',
    'Ac_Water': 'sum',
    'Ac_Land': 'sum',
    'Ac_Unpr': 'sum',
    'D1A' : 'sum',
    'D1B' : 'sum',
    'D1C' : 'sum',
    'D1D' : 'sum',
    'D2A_JPHH':'mean',
    'D3A' : 'sum',
    'D3B' : 'sum',
    'D4A' : 'mean',
    'D4C' : 'mean',
    'D4D' : 'mean',
    'D4E' : 'mean',
    'NatWalkInd':'mean'
}).reset_index()
epa2.head()

,TRACTCE,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,D1A,D1B,D1C,D1D,D2A_JPHH,D3A,D3B,D4A,D4C,D4D,D4E,NatWalkInd
0,900100,3259.741718,1881.326243,1378.415474,657.114690,4.891507,11.697353,1.891265,6.782772,0.671235,20.347963,89.178496,NaN,9.330,12.212073,0.006776,6.416667
1,900201,417.656821,17.664191,399.992631,399.992631,1.737532,5.130095,2.377544,4.115076,1.432229,20.354997,77.363025,84.4900,14.330,22.928422,0.006983,16.833333
2,900202,494.195552,0.101805,494.093747,481.316089,7.106964,23.885063,3.521202,10.628166,0.528865,51.470000,219.297674,313.8200,6.500,20.366152,0.003819,9.555556
3,900203,264.864019,0.000000,264.864019,264.864010,13.232140,39.035883,1.890937,15.123077,0.175281,34.270230,126.340656,540.4700,5.000,24.187664,0.001936,9.333333
4,900300,1370.034583,48.821524,1321.213060,1320.539376,14.057957,31.982802,10.291329,24.349285,0.781902,85.222931,248.081273,697.0475,10.165,22.423071,0.004625,13.208333


In [156]:
# Check missing values
epa2.isna().sum()

TRACTCE        0
Ac_Total       0
Ac_Water       0
Ac_Land        0
Ac_Unpr        0
D1A            0
D1B            0
D1C            0
D1D            0
D2A_JPHH       0
D3A            0
D3B            0
D4A           34
D4C           15
D4D           15
D4E           15
NatWalkInd     0
dtype: int64

In [155]:
epa2.describe()

,TRACTCE,Ac_Total,Ac_Water,Ac_Land,Ac_Unpr,D1A,D1B,D1C,D1D,D2A_JPHH,D3A,D3B,D4A,D4C,D4D,D4E,NatWalkInd
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,49.000000,68.000000,68.000000,68.000000,83.000000
mean,902065.048193,2680.740814,91.922745,2588.818069,2296.124192,6.580494,19.845313,3.242961,9.823455,0.868855,37.524735,146.119055,682.540901,4.202949,9.858711,0.002167,8.575870
std,8679.471376,4356.919446,245.232010,4317.546059,3773.029414,5.390128,15.608313,4.647515,8.499125,1.455196,19.797230,94.071596,267.968634,3.716137,10.431974,0.001830,2.751642
min,900100.000000,224.903007,0.000000,224.903007,224.903007,0.000000,0.000000,0.002279,0.002279,0.000000,2.051457,1.743151,84.490000,0.670000,0.046663,0.000195,4.666667
25%,900953.000000,681.962394,0.774288,676.950218,656.268776,2.800030,8.412513,0.567876,4.100487,0.139096,23.949595,76.060672,514.320000,1.330000,2.857863,0.000759,6.027778
50%,901227.000000,1073.567957,11.933447,1064.918963,1041.306540,4.961676,17.244148,1.311755,6.884898,0.388422,32.689536,130.729751,696.040000,3.332500,5.315784,0.001426,8.250000
75%,901411.500000,2105.420121,43.769935,1976.663477,1839.272619,9.242171,27.828934,3.908175,12.609817,0.768676,49.126872,197.383819,871.283333,5.856875,17.549017,0.003074,10.083333
max,980100.000000,20984.618590,1881.326243,20937.170450,19305.346370,27.543826,78.353423,21.440354,43.502042,8.311246,104.606767,396.031814,1135.930000,18.000000,50.995956,0.007070,16.833333


In [ ]:
epa2.to_csv('df_EPA_TBD.csv',index=False)

### SVI dataset

In [19]:
svi.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,SVI_statewide,AREA_SQMI,E_TOTPOP,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,51,Virginia,VA,51153,Prince William County,51153900100,Census Tract 9001; Prince William County; Virg...,0.1777,2.534004,3234,...,8.4,4.0,0.0,1.2,0.0,1.2,3.5,2.2,5.2,5.5
1,51,Virginia,VA,51153,Prince William County,51153900201,Census Tract 9002.01; Prince William County; V...,0.8996,0.603461,2441,...,3.9,2.2,0.0,1.6,0.0,1.6,3.4,2.8,1.6,1.8
2,51,Virginia,VA,51153,Prince William County,51153900202,Census Tract 9002.02; Prince William County; V...,0.4794,0.772038,4124,...,2.5,1.5,0.0,1.0,0.0,1.0,0.4,0.7,0.0,1.0
3,51,Virginia,VA,51153,Prince William County,51153900203,Census Tract 9002.03; Prince William County; V...,0.9014,0.414283,4589,...,7.5,7.0,0.0,0.9,0.0,0.9,6.8,4.7,0.4,0.7
4,51,Virginia,VA,51153,Prince William County,51153900301,Census Tract 9003.01; Prince William County; V...,0.7464,0.662597,3494,...,15.1,6.0,0.0,1.1,0.0,1.1,3.5,2.3,0.0,1.1


In [20]:
svi.shape

(93, 159)

In [21]:
# Create new variable
svi['E_POV150_P'] = svi['E_POV150']/svi['E_TOTPOP']
svi['E_UNINSUR_P'] = svi['E_UNINSUR']/svi['E_TOTPOP']
svi['E_HBURD_P'] = svi['E_HBURD']/svi['E_HH']
svi['E_SNGPNT_P'] = svi['E_SNGPNT']/svi['E_HH']
svi['E_MUNIT_P'] = svi['E_MUNIT']/svi['E_HU']
svi['E_MOBILE_P'] = svi['E_MOBILE']/svi['E_HU']
svi['E_CROWD_P'] = svi['E_CROWD']/svi['E_HH']
svi['E_NOVEH_P'] = svi['E_NOVEH']/svi['E_HH']
svi['E_GROUPQ_P'] = svi['E_GROUPQ']/svi['E_TOTPOP']

In [22]:
svi_subset = svi[['FIPS','E_TOTPOP','E_HU','E_HH','E_POV150_P','E_UNINSUR_P','E_HBURD_P','E_SNGPNT_P','E_MUNIT_P','E_MOBILE_P',\
                  'E_CROWD_P','E_NOVEH_P','E_GROUPQ_P','SVI_statewide']].rename(columns={'FIPS':'ID'})
svi_subset.head()

,ID,E_TOTPOP,E_HU,E_HH,E_POV150_P,E_UNINSUR_P,E_HBURD_P,E_SNGPNT_P,E_MUNIT_P,E_MOBILE_P,E_CROWD_P,E_NOVEH_P,E_GROUPQ_P,SVI_statewide
0,51153900100,3234,1613,1559,0.031540,0.061843,0.116100,0.007697,0.275883,0.000000,0.004490,0.007056,0.000000,0.1777
1,51153900201,2441,1058,961,0.255633,0.110201,0.387097,0.097815,0.371456,0.000000,0.069719,0.011446,0.002048,0.8996
2,51153900202,4124,1266,1125,0.030310,0.185742,0.193778,0.038222,0.161927,0.004739,0.000000,0.012444,0.000000,0.4794
3,51153900203,4589,1616,1406,0.230115,0.181085,0.343528,0.092461,0.611386,0.000000,0.084637,0.007824,0.001307,0.9014
4,51153900301,3494,1610,1503,0.209502,0.090441,0.275449,0.079175,0.224845,0.000000,0.012641,0.055223,0.000000,0.7464


In [26]:
svi_subset.describe()

,ID,E_TOTPOP,E_HU,E_HH,E_POV150_P,E_UNINSUR_P,E_HBURD_P,E_SNGPNT_P,E_MUNIT_P,E_MOBILE_P,E_CROWD_P,E_NOVEH_P,E_GROUPQ_P,SVI_statewide
count,9.300000e+01,93.000000,93.000000,93.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,5.115390e+10,5173.268817,1705.827957,1641.032258,0.112163,0.101501,0.198070,0.063637,0.121922,0.011039,0.032573,0.023757,0.010593,0.492263
std,8.202184e+03,1635.106273,542.311070,524.889169,0.085251,0.077318,0.112359,0.044061,0.165814,0.031604,0.033907,0.023866,0.049755,0.303724
min,5.115390e+10,0.000000,0.000000,0.000000,0.000000,0.006809,0.028169,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007400
25%,5.115390e+10,4070.000000,1327.000000,1293.000000,0.044693,0.044687,0.103865,0.029747,0.000000,0.000000,0.004991,0.006870,0.000000,0.207625
50%,5.115390e+10,5165.000000,1703.000000,1609.000000,0.091423,0.079091,0.188528,0.055061,0.041129,0.000000,0.024324,0.014498,0.000000,0.525000
75%,5.115390e+10,6419.000000,2126.000000,1995.000000,0.167946,0.135911,0.274228,0.087933,0.177470,0.004339,0.048249,0.036158,0.001711,0.787000
max,5.115398e+10,9046.000000,3015.000000,2931.000000,0.384721,0.391522,0.540000,0.175745,0.643294,0.168823,0.144811,0.127924,0.404803,0.990300


In [27]:
svi_subset.isna().sum()

ID               0
E_TOTPOP         0
E_HU             0
E_HH             0
E_POV150_P       1
E_UNINSUR_P      1
E_HBURD_P        1
E_SNGPNT_P       1
E_MUNIT_P        1
E_MOBILE_P       1
E_CROWD_P        1
E_NOVEH_P        1
E_GROUPQ_P       1
SVI_statewide    1
dtype: int64

In [24]:
# Inner join
df = df.merge(svi_subset,on='ID',how='inner')
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,E_POV150_P,E_UNINSUR_P,E_HBURD_P,E_SNGPNT_P,E_MUNIT_P,E_MOBILE_P,E_CROWD_P,E_NOVEH_P,E_GROUPQ_P,SVI_statewide
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.031540,0.061843,0.116100,0.007697,0.275883,0.000000,0.004490,0.007056,0.000000,0.1777
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.255633,0.110201,0.387097,0.097815,0.371456,0.000000,0.069719,0.011446,0.002048,0.8996
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.030310,0.185742,0.193778,0.038222,0.161927,0.004739,0.000000,0.012444,0.000000,0.4794
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.230115,0.181085,0.343528,0.092461,0.611386,0.000000,0.084637,0.007824,0.001307,0.9014
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.209502,0.090441,0.275449,0.079175,0.224845,0.000000,0.012641,0.055223,0.000000,0.7464


In [25]:
df.shape

(93, 36)

### ACS

#### Vacancy House: Vacancy -- Percentage of housing units that were vacant

In [32]:
vacancyhouse

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"1,613",±182,"1,058",±107,"1,266",±164,"1,616",±251,"1,610",...,"1,285",±196,711,±60,"2,126",±204,"2,378",±204,0,±13
1,Occupied,"1,559",±185,961,±115,"1,125",±159,"1,406",±293,"1,503",...,"1,243",±193,672,±75,"1,937",±188,"2,317",±186,0,±13
2,Vacant,54,±61,97,±77,141,±106,210,±111,107,...,42,±53,39,±41,189,±139,61,±83,0,±13


In [43]:
# Keep only columns containing "Estimate"
df_filtered = vacancyhouse.loc[:, vacancyhouse.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column
df_long.rename(columns={"index": "Census_Tract",\
                        0:'Total',
                        1:'Occupied',
                        2:'Vacant'}, inplace=True)

# Convert Census_Tract to numeric
df_long["Census_Tract"] = pd.to_numeric(df_long["Census_Tract"])
df_long[['Total', 'Occupied', 'Vacant']] = df_long[['Total', 'Occupied', 'Vacant']].replace(',', '', regex=True).astype(int)
df_long['House_Vacant_P'] = df_long['Vacant']/df_long['Total']
df_long.head()

,Census_Tract,Total,Occupied,Vacant,House_Vacant_P
0,9001,1613,1559,54,0.033478
1,9002,1058,961,97,0.091682
2,9002,1266,1125,141,0.111374
3,9002,1616,1406,210,0.129950
4,9003,1610,1503,107,0.066460
...,...,...,...,...,...
88,9017,1285,1243,42,0.032685
89,9017,711,672,39,0.054852
90,9017,2126,1937,189,0.088899
91,9019,2378,2317,61,0.025652


In [48]:
df = pd.concat([df,df_long['House_Vacant_P']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,E_UNINSUR_P,E_HBURD_P,E_SNGPNT_P,E_MUNIT_P,E_MOBILE_P,E_CROWD_P,E_NOVEH_P,E_GROUPQ_P,SVI_statewide,House_Vacant_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.061843,0.116100,0.007697,0.275883,0.000000,0.004490,0.007056,0.000000,0.1777,0.033478
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.110201,0.387097,0.097815,0.371456,0.000000,0.069719,0.011446,0.002048,0.8996,0.091682
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.185742,0.193778,0.038222,0.161927,0.004739,0.000000,0.012444,0.000000,0.4794,0.111374
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.181085,0.343528,0.092461,0.611386,0.000000,0.084637,0.007824,0.001307,0.9014,0.129950
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.090441,0.275449,0.079175,0.224845,0.000000,0.012641,0.055223,0.000000,0.7464,0.066460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.320190,0.222043,0.086082,0.000000,0.157198,0.144811,0.065165,0.000000,0.9107,0.032685
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.203474,0.221726,0.020833,0.000000,0.000000,0.000000,0.031250,0.000000,0.2545,0.054852
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.391522,0.375839,0.075374,0.362653,0.000000,0.069179,0.044399,0.000760,0.9426,0.088899
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.260060,0.318084,0.157100,0.373844,0.004205,0.068192,0.014243,0.001164,0.9130,0.025652


#### Medicaid: Percentage of the population with Medicaid as their health insurance coverage. Medicaid is public health insurance for people with low incomes or a disability.

In [58]:
medicaid.head()

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"3,159",±423,"2,410",±453,"4,108",±814,"4,574",±981,"3,462",...,"4,593",±784,"2,418",±333,"6,581","±1,263","7,696",±939,0,±13
1,Male:,"1,555",±251,"1,075",±214,"2,217",±494,"1,920",±383,"1,665",...,"2,251",±457,"1,228",±212,"4,018",±822,"4,019",±707,0,±13
2,Under 19 years:,171,±70,260,±156,416,±210,450,±158,405,...,639,±224,348,±70,"1,170",±398,"1,228",±377,0,±13
3,With Medicaid/means-tested public ...,47,±48,169,±140,31,±37,200,±150,176,...,249,±137,142,±75,478,±278,504,±308,0,±13
4,No Medicaid/means-tested public co...,124,±72,91,±60,385,±193,250,±118,229,...,390,±186,206,±96,692,±367,724,±375,0,±13


In [62]:
# Keep only columns containing "Estimate"
df_filtered = medicaid.loc[:, medicaid.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T 
df_long.reset_index(inplace=True)
df_long = df_long[['index',0,3,6,9,13,16,19]]

# Transform variables
df_long[[0,3,6,9,13,16,19]] = df_long[[0,3,6,9,13,16,19]].replace(',', '', regex=True).astype(int)
df_long['with_med'] = df_long[3]+df_long[6]+df_long[9]+ df_long[13]+df_long[16]+df_long[19]
df_long['With_Medicaid_P'] = df_long['with_med']/df_long[0]
df_long.head()

,index,0,3,6,9,13,16,19,with_med,With_Medicaid_P
0,9001,3159,47,22,33,47,49,24,222,0.070275
1,9002,2410,169,45,0,120,130,8,472,0.195851
2,9002,4108,31,35,20,378,169,4,637,0.155063
3,9002,4574,200,192,58,415,168,0,1033,0.225842
4,9003,3462,176,0,9,129,268,14,596,0.172155
...,...,...,...,...,...,...,...,...,...,...
88,9017,4593,249,98,13,356,37,25,778,0.169388
89,9017,2418,142,45,4,141,47,0,379,0.156741
90,9017,6581,478,12,110,375,131,114,1220,0.185382
91,9019,7696,504,225,41,449,142,114,1475,0.191658


In [59]:
df = pd.concat([df,df_long['With_Medicaid_P']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,E_HBURD_P,E_SNGPNT_P,E_MUNIT_P,E_MOBILE_P,E_CROWD_P,E_NOVEH_P,E_GROUPQ_P,SVI_statewide,House_Vacant_P,With_Medicaid_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.116100,0.007697,0.275883,0.000000,0.004490,0.007056,0.000000,0.1777,0.033478,0.070275
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.387097,0.097815,0.371456,0.000000,0.069719,0.011446,0.002048,0.8996,0.091682,0.195851
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.193778,0.038222,0.161927,0.004739,0.000000,0.012444,0.000000,0.4794,0.111374,0.155063
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.343528,0.092461,0.611386,0.000000,0.084637,0.007824,0.001307,0.9014,0.129950,0.225842
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.275449,0.079175,0.224845,0.000000,0.012641,0.055223,0.000000,0.7464,0.066460,0.172155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.222043,0.086082,0.000000,0.157198,0.144811,0.065165,0.000000,0.9107,0.032685,0.169388
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.221726,0.020833,0.000000,0.000000,0.000000,0.031250,0.000000,0.2545,0.054852,0.156741
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.375839,0.075374,0.362653,0.000000,0.069179,0.044399,0.000760,0.9426,0.088899,0.185382
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.318084,0.157100,0.373844,0.004205,0.068192,0.014243,0.001164,0.9130,0.025652,0.191658


#### transportationmode (Estimated percentage of workers who drive/use public transit/bicycle/walk/stay home to work)

In [65]:
transportationmode.head()

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"1,710",±369,"1,497",±267,"2,431",±537,"1,972",±525,"1,815",...,"2,111",±351,"1,357",±247,"3,325",±915,"4,196",±569,0,±13
1,Total:No vehicle available,11,±17,22,±34,12,±21,0,±13,103,...,108,±140,39,±63,0,±19,27,±39,0,±13
2,Total:1 vehicle available,347,±106,367,±96,125,±85,511,±183,549,...,339,±251,143,±96,"1,414",±601,"1,282",±489,0,±13
3,Total:2 vehicles available,976,±369,577,±238,606,±226,740,±308,815,...,773,±280,293,±154,"1,291",±618,"1,322",±533,0,±13
4,Total:3 or more vehicles available,376,±161,531,±231,"1,688",±591,721,±461,348,...,891,±281,882,±312,620,±426,"1,565",±536,0,±13


In [64]:
# Keep only columns containing "Estimate"
df_filtered = transportationmode.loc[:, transportationmode.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T 
df_long.reset_index(inplace=True)

# Transform variables
df_long = df_long[['index',0,5,10,15,20,25,30]]
df_long[[0,5,10,15,20,25,30]] = df_long[[0,5,10,15,20,25,30]].replace(',', '', regex=True).astype(int)
df_long['Work_Drivealone_P'] = df_long[5]/df_long[0]
df_long['Work_Carpooled_P'] = df_long[10]/df_long[0]
df_long['Work_PublicTransportation_P'] = df_long[15]/df_long[0]
df_long['Work_Walk_P'] = df_long[20]/df_long[0]
df_long['Work_Taximotorbike_P'] = df_long[25]/df_long[0]
df_long['Work_Fromhome_P'] = df_long[30]/df_long[0]
df_long.head()

,index,0,5,10,15,20,25,30,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P
0,9001,1710,1051,44,89,0,128,398,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749
1,9002,1497,1023,161,25,23,0,265,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021
2,9002,2431,1840,308,24,0,98,161,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228
3,9002,1972,1513,160,38,122,71,68,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483
4,9003,1815,1120,162,132,139,16,246,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,9017,2111,1497,280,60,119,8,147,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635
89,9017,1357,1029,251,21,0,31,25,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423
90,9017,3325,2024,724,0,228,0,349,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962
91,9019,4196,3215,605,9,0,15,352,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889


In [67]:
df = pd.concat([df,df_long[['Work_Drivealone_P','Work_Carpooled_P','Work_PublicTransportation_P','Work_Walk_P','Work_Taximotorbike_P','Work_Fromhome_P']]],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,E_GROUPQ_P,SVI_statewide,House_Vacant_P,With_Medicaid_P,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.000000,0.1777,0.033478,0.070275,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.002048,0.8996,0.091682,0.195851,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.000000,0.4794,0.111374,0.155063,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.001307,0.9014,0.129950,0.225842,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.000000,0.7464,0.066460,0.172155,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.000000,0.9107,0.032685,0.169388,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.000000,0.2545,0.054852,0.156741,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.000760,0.9426,0.088899,0.185382,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.001164,0.9130,0.025652,0.191658,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889


#### snap

In [70]:
snap

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"1,559",±185,961,±115,"1,125",±159,"1,406",±293,"1,503",...,"1,243",±193,672,±75,"1,937",±188,"2,317",±186,0,±13
1,With cash public assistance or Food Stamps...,32,±35,77,±59,138,±144,107,±62,133,...,68,±36,53,±44,147,±108,110,±114,0,±13
2,No cash public assistance or Food Stamps/SNAP,"1,527",±188,884,±125,987,±142,"1,299",±300,"1,370",...,"1,175",±193,619,±86,"1,790",±194,"2,207",±180,0,±13


In [71]:
# Keep only columns containing "Estimate"
df_filtered = snap.loc[:, snap.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column
df_long.rename(columns={"index": "Census_Tract",\
                        0:'Total',
                        1:'With',
                        2:'No'}, inplace=True)

# Convert Census_Tract to numeric
df_long["Census_Tract"] = pd.to_numeric(df_long["Census_Tract"])
df_long[['Total', 'With', 'No']] = df_long[['Total', 'With', 'No']].replace(',', '', regex=True).astype(int)
df_long['With_PublicAssIncome_P'] = df_long['With']/df_long['Total']
df_long.head()

,Census_Tract,Total,With,No,With_PublicAssIncome_P
0,9001,1559,32,1527,0.020526
1,9002,961,77,884,0.080125
2,9002,1125,138,987,0.122667
3,9002,1406,107,1299,0.076102
4,9003,1503,133,1370,0.088490
...,...,...,...,...,...
88,9017,1243,68,1175,0.054706
89,9017,672,53,619,0.078869
90,9017,1937,147,1790,0.075891
91,9019,2317,110,2207,0.047475


In [72]:
df = pd.concat([df,df_long['With_PublicAssIncome_P']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,SVI_statewide,House_Vacant_P,With_Medicaid_P,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.1777,0.033478,0.070275,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.8996,0.091682,0.195851,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.4794,0.111374,0.155063,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.9014,0.129950,0.225842,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.7464,0.066460,0.172155,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.9107,0.032685,0.169388,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635,0.054706
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.2545,0.054852,0.156741,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423,0.078869
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.9426,0.088899,0.185382,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962,0.075891
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.9130,0.025652,0.191658,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889,0.047475


#### SSI

In [74]:
ssi

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"1,559",±185,961,±115,"1,125",±159,"1,406",±293,"1,503",...,"1,243",±193,672,±75,"1,937",±188,"2,317",±186,0,±13
1,With Supplemental Security Income (SSI),83,±51,36,±27,0,±13,109,±67,35,...,48,±34,23,±24,63,±63,89,±76,0,±13
2,No Supplemental Security Income (SSI),"1,476",±192,925,±116,"1,125",±159,"1,297",±286,"1,468",...,"1,195",±192,649,±73,"1,874",±190,"2,228",±201,0,±13


In [75]:
# Keep only columns containing "Estimate"
df_filtered = ssi.loc[:, ssi.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column
df_long.rename(columns={"index": "Census_Tract",\
                        0:'Total',
                        1:'With',
                        2:'No'}, inplace=True)

# Convert Census_Tract to numeric
df_long["Census_Tract"] = pd.to_numeric(df_long["Census_Tract"])
df_long[['Total', 'With', 'No']] = df_long[['Total', 'With', 'No']].replace(',', '', regex=True).astype(int)
df_long['With_SSI_P'] = df_long['With']/df_long['Total']
df_long.head()

,Census_Tract,Total,With,No,With_SSI_P
0,9001,1559,83,1476,0.053239
1,9002,961,36,925,0.037461
2,9002,1125,0,1125,0.000000
3,9002,1406,109,1297,0.077525
4,9003,1503,35,1468,0.023287
...,...,...,...,...,...
88,9017,1243,48,1195,0.038616
89,9017,672,23,649,0.034226
90,9017,1937,63,1874,0.032525
91,9019,2317,89,2228,0.038412


In [76]:
df = pd.concat([df,df_long['With_SSI_P']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,House_Vacant_P,With_Medicaid_P,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P,With_SSI_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.033478,0.070275,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526,0.053239
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.091682,0.195851,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125,0.037461
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.111374,0.155063,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667,0.000000
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.129950,0.225842,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102,0.077525
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.066460,0.172155,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490,0.023287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.032685,0.169388,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635,0.054706,0.038616
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.054852,0.156741,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423,0.078869,0.034226
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.088899,0.185382,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962,0.075891,0.032525
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.025652,0.191658,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889,0.047475,0.038412


#### travel time

In [78]:
traveltime

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"1,312",±363,"1,232",±191,"2,270",±476,"1,907",±527,"1,569",...,"1,964",±329,"1,332",±245,"2,978",±729,"3,846",±558,0,±13
1,Less than 5 minutes,0,±13,15,±25,0,±13,0,±13,7,...,70,±101,52,±77,228,±308,0,±19,0,±13
2,5 to 9 minutes,8,±13,43,±40,160,±105,18,±21,60,...,92,±89,197,±109,196,±154,74,±71,0,±13
3,10 to 14 minutes,37,±44,107,±67,233,±129,323,±186,162,...,104,±56,153,±102,190,±192,380,±224,0,±13
4,15 to 19 minutes,44,±36,87,±52,227,±137,284,±142,188,...,379,±190,80,±81,635,±339,662,±400,0,±13
5,20 to 24 minutes,211,±185,76,±42,188,±108,52,±45,90,...,157,±78,124,±61,256,±158,646,±281,0,±13
6,25 to 29 minutes,32,±29,39,±36,158,±131,49,±61,55,...,51,±48,67,±54,157,±146,344,±204,0,±13
7,30 to 34 minutes,162,±114,190,±71,178,±111,394,±252,371,...,160,±80,172,±103,274,±178,243,±129,0,±13
8,35 to 39 minutes,40,±49,114,±55,283,±178,31,±28,59,...,46,±44,142,±65,26,±29,72,±72,0,±13
9,40 to 44 minutes,129,±70,156,±70,143,±140,21,±24,81,...,201,±140,19,±30,244,±193,404,±257,0,±13


In [80]:
# Keep only columns containing "Estimate"
df_filtered = traveltime.loc[:, traveltime.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column

# Transform variables
df_long[[0,1,2,3,4,5,6,7,8,9,10,11,12]] = df_long[[0,1,2,3,4,5,6,7,8,9,10,11,12]].replace(',', '', regex=True).astype(int)
df_long['Mean_Transportation_time(min)'] = (df_long[1]*2.5+df_long[2]*7.5+df_long[3]*12.5+df_long[4]*17.5+df_long[5]*22.5+df_long[6]*27.5+\
                                            df_long[7]*32.5+df_long[8]*37.5+df_long[9]*42.5+df_long[10]*52.5+df_long[11]*75+df_long[12]*100)/df_long[0]
df_long.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,Mean_Transportation_time(min)
0,9001,1312,0,8,37,44,211,32,162,40,129,427,163,59,45.510671
1,9002,1232,15,43,107,87,76,39,190,114,156,236,82,87,40.846185
2,9002,2270,0,160,233,227,188,158,178,283,143,471,222,7,35.776432
3,9002,1907,0,18,323,284,52,49,394,31,21,345,337,53,39.437598
4,9003,1569,7,60,162,188,90,55,371,59,81,285,165,46,37.584449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,9017,1964,70,92,104,379,157,51,160,46,201,316,332,56,38.844196
89,9017,1332,52,197,153,80,124,67,172,142,19,209,17,100,32.674550
90,9017,2978,228,196,190,635,256,157,274,26,244,253,433,86,33.650940
91,9019,3846,0,74,380,662,646,344,243,72,404,321,561,139,36.786271


In [81]:
df = pd.concat([df,df_long['Mean_Transportation_time(min)']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,With_Medicaid_P,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P,With_SSI_P,Mean_Transportation_time(min)
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.070275,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526,0.053239,45.510671
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.195851,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125,0.037461,40.846185
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.155063,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667,0.000000,35.776432
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.225842,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102,0.077525,39.437598
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.172155,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490,0.023287,37.584449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.169388,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635,0.054706,0.038616,38.844196
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.156741,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423,0.078869,0.034226,32.674550
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.185382,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962,0.075891,0.032525,33.650940
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.191658,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889,0.047475,0.038412,36.786271


#### Housing Cost Burden (Proportion of households for which selected monthly costs.)

In [83]:
housingcost

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,233,±96,518,±98,169,±72,"1,277",±295,541,...,475,±212,8,±15,"1,238",±228,"1,201",±218,0,±13
1,Less than 10.0 percent,9,±14,19,±20,0,±13,0,±13,14,...,19,±32,0,±13,0,±19,21,±37,0,±13
2,10.0 to 14.9 percent,53,±52,60,±48,26,±41,42,±43,73,...,5,±22,0,±13,74,±113,128,±138,0,±13
3,15.0 to 19.9 percent,81,±60,49,±62,0,±13,340,±263,39,...,155,±120,0,±13,188,±147,349,±195,0,±13
4,20.0 to 24.9 percent,17,±19,40,±32,38,±36,37,±48,66,...,0,±13,0,±13,199,±155,40,±46,0,±13
5,25.0 to 29.9 percent,16,±24,30,±28,0,±13,382,±129,123,...,107,±167,0,±13,52,±49,90,±99,0,±13
6,30.0 to 34.9 percent,9,±14,59,±47,31,±46,58,±40,21,...,52,±50,0,±13,30,±35,228,±173,0,±13
7,35.0 to 39.9 percent,0,±13,43,±43,0,±13,7,±14,12,...,58,±55,0,±13,255,±173,166,±182,0,±13
8,40.0 to 49.9 percent,11,±16,95,±59,14,±22,62,±46,27,...,17,±29,8,±15,146,±89,66,±69,0,±13
9,50.0 percent or more,25,±37,110,±64,0,±13,338,±176,161,...,22,±28,0,±13,198,±121,95,±90,0,±13


In [85]:
# Keep only columns containing "Estimate"
df_filtered = housingcost.loc[:, housingcost.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column

# Transform variables
df_long[[0,1,2,3,4,5,6,7,8,9,10]] = df_long[[0,1,2,3,4,5,6,7,8,9,10]].replace(',', '', regex=True).astype(int)
df_long[11] = df_long[0]-df_long[10]
df_long['Mean_Proportion_HHIncome'] = (df_long[1]*0.05+df_long[2]*0.125+df_long[3]*0.175+df_long[4]*0.225+df_long[5]*0.275+df_long[6]*0.325+\
                                            df_long[7]*0.375+df_long[8]*0.45+df_long[9]*0.60)/df_long[11]
df_long.head()

,index,0,1,2,3,4,5,6,7,8,9,10,11,Mean_Proportion_HHIncome
0,9001,233,9,53,81,17,16,9,0,11,25,12,221,0.236878
1,9002,518,19,60,49,40,30,59,43,95,110,13,505,0.353119
2,9002,169,0,26,0,38,0,31,0,14,0,60,109,0.258486
3,9002,1277,0,42,340,37,382,58,7,62,338,11,1266,0.339889
4,9003,541,14,73,39,66,123,21,12,27,161,5,536,0.345896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,9017,475,19,5,155,0,107,52,58,17,22,40,435,0.270402
89,9017,8,0,0,0,0,0,0,0,8,0,0,8,0.450000
90,9017,1238,0,74,188,199,52,30,255,146,198,96,1142,0.342469
91,9019,1201,21,128,349,40,90,228,166,66,95,18,1183,0.283115


In [87]:
df = pd.concat([df,df_long['Mean_Proportion_HHIncome']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,Work_Drivealone_P,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P,With_SSI_P,Mean_Transportation_time(min),Mean_Proportion_HHIncome
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.614620,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526,0.053239,45.510671,0.236878
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.683367,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125,0.037461,40.846185,0.353119
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.756890,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667,0.000000,35.776432,0.258486
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.767241,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102,0.077525,39.437598,0.339889
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.617080,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490,0.023287,37.584449,0.345896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.709143,0.132639,0.028423,0.056371,0.003790,0.069635,0.054706,0.038616,38.844196,0.270402
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.758290,0.184967,0.015475,0.000000,0.022845,0.018423,0.078869,0.034226,32.674550,0.450000
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.608722,0.217744,0.000000,0.068571,0.000000,0.104962,0.075891,0.032525,33.650940,0.342469
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.766206,0.144185,0.002145,0.000000,0.003575,0.083889,0.047475,0.038412,36.786271,0.283115


#### populationoccupied (Total Population in Occupied Housing Units by Tenure)

In [89]:
populationoccupied

,Label (Grouping),Census Tract 9001; Prince William County; Virginia!!Estimate,Census Tract 9001; Prince William County; Virginia!!Margin of Error,Census Tract 9002.01; Prince William County; Virginia!!Estimate,Census Tract 9002.01; Prince William County; Virginia!!Margin of Error,Census Tract 9002.02; Prince William County; Virginia!!Estimate,Census Tract 9002.02; Prince William County; Virginia!!Margin of Error,Census Tract 9002.03; Prince William County; Virginia!!Estimate,Census Tract 9002.03; Prince William County; Virginia!!Margin of Error,Census Tract 9003.01; Prince William County; Virginia!!Estimate,...,Census Tract 9017.02; Prince William County; Virginia!!Estimate,Census Tract 9017.02; Prince William County; Virginia!!Margin of Error,Census Tract 9017.03; Prince William County; Virginia!!Estimate,Census Tract 9017.03; Prince William County; Virginia!!Margin of Error,Census Tract 9017.04; Prince William County; Virginia!!Estimate,Census Tract 9017.04; Prince William County; Virginia!!Margin of Error,Census Tract 9019; Prince William County; Virginia!!Estimate,Census Tract 9019; Prince William County; Virginia!!Margin of Error,Census Tract 9801; Prince William County; Virginia!!Estimate,Census Tract 9801; Prince William County; Virginia!!Margin of Error
0,Total:,"3,234",±442,"2,436",±455,"4,124",±816,"4,583",±971,"3,494",...,"4,641",±793,"2,418",±333,"6,577","±1,264","7,720",±935,0,±13
1,Owner occupied,"2,717",±442,"1,293",±337,"3,472",±816,466,±231,"2,110",...,"2,864",±657,"2,394",±328,"2,787",±783,"3,529",±846,0,±13
2,Renter occupied,517,±232,"1,143",±349,652,±326,"4,117",±955,"1,384",...,"1,777",±635,24,±44,"3,790","±1,330","4,191",±946,0,±13


In [90]:
# Keep only columns containing "Estimate"
df_filtered = populationoccupied.loc[:, populationoccupied.columns.str.contains("Estimate")]

# Extract only the Census Tract number using regex
df_filtered.columns = [re.search(r'Census Tract (\d+)', col).group(1) for col in df_filtered.columns]

# Convert Wide to Long Format
df_long = df_filtered.T  
df_long.reset_index(inplace=True)  # Reset index to make it a column
df_long.rename(columns={"index": "Census_Tract",\
                        0:'Total',
                        1:'Owner',
                        2:'Renter'}, inplace=True)

# Convert Census_Tract to numeric
df_long["Census_Tract"] = pd.to_numeric(df_long["Census_Tract"])
df_long[['Total', 'Owner', 'Renter']] = df_long[['Total', 'Owner', 'Renter']].replace(',', '', regex=True).astype(int)
df_long['Owner_occupied_P'] = df_long['Owner']/df_long['Total']
df_long.head()

,Census_Tract,Total,Owner,Renter,Owner_occupied_P
0,9001,3234,2717,517,0.840136
1,9002,2436,1293,1143,0.530788
2,9002,4124,3472,652,0.841901
3,9002,4583,466,4117,0.101680
4,9003,3494,2110,1384,0.603892
...,...,...,...,...,...
88,9017,4641,2864,1777,0.617108
89,9017,2418,2394,24,0.990074
90,9017,6577,2787,3790,0.423749
91,9019,7720,3529,4191,0.457124


In [91]:
df = pd.concat([df,df_long['Owner_occupied_P']],axis=1)
df.head()

,ID,PEOPCOLORPCT,LOWINCPCT,UNEMPPCT,DISABILITYPCT,LINGISOPCT,LESSHSPCT,UNDER5PCT,OVER64PCT,LIFEEXPPCT,...,Work_Carpooled_P,Work_PublicTransportation_P,Work_Walk_P,Work_Taximotorbike_P,Work_Fromhome_P,With_PublicAssIncome_P,With_SSI_P,Mean_Transportation_time(min),Mean_Proportion_HHIncome,Owner_occupied_P
0,51153900100,0.501237,0.076685,0.006587,0.095916,0.033355,0.044565,0.057205,0.299320,0.182564,...,0.025731,0.052047,0.000000,0.074854,0.232749,0.020526,0.053239,45.510671,0.236878,0.840136
1,51153900201,0.771815,0.327176,0.067423,0.154357,0.096774,0.133784,0.032364,0.072102,0.217436,...,0.107548,0.016700,0.015364,0.000000,0.177021,0.080125,0.037461,40.846185,0.353119,0.530788
2,51153900202,0.731086,0.276916,0.048494,0.039679,0.135111,0.171480,0.040980,0.066440,0.169231,...,0.126697,0.009872,0.000000,0.040313,0.066228,0.122667,0.000000,35.776432,0.258486,0.841901
3,51153900203,0.889301,0.301155,0.042468,0.100787,0.143670,0.191298,0.095228,0.057529,0.206154,...,0.081136,0.019270,0.061866,0.036004,0.034483,0.076102,0.077525,39.437598,0.339889,0.101680
4,51153900301,0.556382,0.232398,0.092181,0.134315,0.093147,0.109264,0.048369,0.089010,0.180513,...,0.089256,0.072727,0.076584,0.008815,0.135537,0.088490,0.023287,37.584449,0.345896,0.603892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,51153901702,0.744667,0.337302,0.121313,0.111039,0.181014,0.350799,0.054730,0.077785,0.226667,...,0.132639,0.028423,0.056371,0.003790,0.069635,0.054706,0.038616,38.844196,0.270402,0.617108
89,51153901703,0.682796,0.236175,0.000000,0.055418,0.000000,0.249219,0.038462,0.097188,0.185641,...,0.184967,0.015475,0.000000,0.022845,0.018423,0.078869,0.034226,32.674550,0.450000,0.990074
90,51153901704,0.890003,0.364426,0.068152,0.090108,0.318534,0.247506,0.109693,0.112124,0.185641,...,0.217744,0.000000,0.068571,0.000000,0.104962,0.075891,0.032525,33.650940,0.342469,0.423749
91,51153901900,0.779273,0.262259,0.028372,0.058732,0.123004,0.230222,0.095096,0.074136,0.191795,...,0.144185,0.002145,0.000000,0.003575,0.083889,0.047475,0.038412,36.786271,0.283115,0.457124


In [92]:
df.to_csv('Clean_df_safetyindex.csv',index=False)

### Discrepancy Examine

In [ ]:
epa['TRACT'] = '51153' + epa['TRACTCE'].astype(str)  
epa['TRACT'] = epa['TRACT'].astype(int)  
epa['TRACT'].nunique()

In [ ]:
# Convert both unique values to lists
epa_tract_list = epa['TRACT'].unique().tolist()
df_id_list = df['ID'].unique().tolist()

# Find differences
only_in_epa = set(epa_tract_list) - set(df_id_list)
only_in_df = set(df_id_list) - set(epa_tract_list)
common_values = set(epa_tract_list) & set(df_id_list)

# Print results
print(f"Values in EPA but not in DF: {only_in_epa}")
print(f"Values in DF but not in EPA: {only_in_df}")
print(f"Common Values: {len(common_values)}")

Several tract id need to be examined: 

- 900301/900302 -- 900300
- 900601/900602 -- 900600
- 900803/900804 -- 900802
- 901703/901704 -- 901701
- 901101/901102 -- 901100
- 901013/901014/901015/901016
- 900503/900504 -- 900501
- 901418/901419/901420/901421

Unknown: 901408/901410/901008/901001